# Aggregate responses

Aggregate the synthetic person-level data to find the proportion who gave each response to each question.

## Set-up

### Packages and file paths

In [1]:
# Import required packages
from collections import defaultdict
from dataclasses import dataclass
import numpy as np
import os
import pandas as pd

In [2]:
# File paths
@dataclass(frozen=True)
class Paths:
    '''Stores paths to data and files'''
    survey = '../data/survey_data'
    synthetic_data = 'synthetic_data_raw.csv'
    aggregate = 'aggregate_responses.csv'


paths = Paths()

In [3]:
# Import functions defined elsewhere
import sys
sys.path.append('../')
from utilities.response_labels import create_response_label_dict

### Import raw data

In [4]:
data = pd.read_csv(os.path.join(paths.survey, paths.synthetic_data))
data.head()

,gender,transgender,sexual_orientation,neurodivergent,birth_parent1,birth_parent2,birth_you,birth_you_age,autonomy_pressure,autonomy_express,...,peer_talk_listen_lab,peer_talk_helpful_lab,peer_talk_if_lab,accept_peer_lab,year_group_lab,fsm_lab,sen_lab,ethnicity_lab,english_additional_lab,school_lab
0,4.0,2.0,6.0,3.0,2.0,1.0,2.0,NaN,2.0,5.0,...,Fully,Somewhat helpful,Very uncomfortable,Not at all,Year 10,Non-FSM,Non-SEN,Ethnic minority,No,School E
1,NaN,2.0,1.0,3.0,3.0,2.0,3.0,8.0,4.0,2.0,...,Mostly,Very helpful,Very uncomfortable,Slightly,Year 10,Non-FSM,Non-SEN,NaN,No,School D
2,NaN,3.0,4.0,1.0,1.0,1.0,1.0,1.0,5.0,4.0,...,Mostly,Very helpful,Very comfortable,Not at all,Year 10,Non-FSM,NaN,NaN,NaN,School E
3,NaN,5.0,5.0,2.0,2.0,2.0,1.0,3.0,1.0,2.0,...,Fully,Somewhat helpful,Uncomfortable,Mostly,Year 10,Non-FSM,Non-SEN,White British,No,School G
4,5.0,3.0,4.0,1.0,1.0,3.0,NaN,2.0,5.0,2.0,...,Slightly,Somewhat helpful,Uncomfortable,Not at all,Year 8,NaN,Non-SEN,White British,Yes,NaN


## Find the proportion giving each response to each measure, within a given group

In [5]:
# Make list of columns that we want to count responses for
response_col = [col for col in data.columns if (
    col.endswith('_lab') and col not in [
        'school_lab', 'gender_lab', 'transgender_lab', 'sexual_orientation_lab',
        'neurodivergent_lab', 'birth_parent1_lab', 'birth_parent2_lab',
        'birth_you_lab', 'birth_you_age_lab', 'year_group_lab', 'fsm_lab',
        'sen_lab', 'ethnicity_lab', 'english_additional_lab'])]
response_col

['autonomy_pressure_lab',
 'autonomy_express_lab',
 'autonomy_decide_lab',
 'autonomy_told_lab',
 'autonomy_myself_lab',
 'autonomy_choice_lab',
 'life_satisfaction_lab',
 'optimism_future_lab',
 'optimism_best_lab',
 'optimism_good_lab',
 'optimism_work_lab',
 'wellbeing_optimistic_lab',
 'wellbeing_useful_lab',
 'wellbeing_relaxed_lab',
 'wellbeing_problems_lab',
 'wellbeing_thinking_lab',
 'wellbeing_close_lab',
 'wellbeing_mind_lab',
 'esteem_satisfied_lab',
 'esteem_qualities_lab',
 'esteem_well_lab',
 'esteem_value_lab',
 'esteem_good_lab',
 'stress_control_lab',
 'stress_overcome_lab',
 'stress_confident_lab',
 'stress_way_lab',
 'appearance_happy_lab',
 'appearance_feel_lab',
 'negative_lonely_lab',
 'negative_unhappy_lab',
 'negative_like_lab',
 'negative_cry_lab',
 'negative_school_lab',
 'negative_worry_lab',
 'negative_sleep_lab',
 'negative_wake_lab',
 'negative_shy_lab',
 'negative_scared_lab',
 'lonely_lab',
 'support_ways_lab',
 'support_look_lab',
 'sleep_lab',
 'physi

In [6]:
# Import dictionary which contains the response options for each question,
# for which we want to know the answers to
labels = create_response_label_dict()

# Add 'NaN': 'No response' to each of the dictionaries
# They are stored as dictionary of dictionaries, so we loop through and
# update each one
for key, value in labels.items():
    value.update({np.nan: 'No response'})

# Preview dictionary
labels

{'gender': {1: 'Girl',
  2: 'Boy',
  3: 'Non-binary',
  4: 'I describe myself in another way',
  5: 'Currently unsure',
  6: 'Prefer not to say',
  nan: 'No response'},
 'transgender': {1: 'Yes',
  2: 'No',
  3: 'Prefer not to say',
  4: 'I describe myself in another way',
  5: 'Not sure',
  nan: 'No response'},
 'sexual_orientation': {1: 'Bi/pansexual',
  2: 'Gay/lesbian',
  3: 'Heterosexual/straight',
  4: 'I describe myself in another way',
  5: 'Currently unsure',
  6: 'Prefer not to say',
  nan: 'No response'},
 'neurodivergent': {1: 'Yes', 2: 'No', 3: 'Unsure', nan: 'No response'},
 'birth': {1: 'Yes', 2: 'No', 3: "I don't know", nan: 'No response'},
 'birth_you_age': {1: 'Under 1 year old',
  2: '1 year old',
  3: '2 years old',
  4: '3 years old',
  5: '4 years old',
  6: '5 years old',
  7: '6 years old',
  8: '7 years old',
  9: '8 years old',
  10: '9 years old',
  11: '10 years old',
  12: '11 years old',
  13: '12 years old',
  14: '13 years old',
  15: '14 years old',
  1

Define function for aggregating responses to each variable, designed so that it counts the occurence of all possible values, rather than basing it on only the values present (e.g. if e.g. no-one responded 3, it will just have counts of responses to 1 2 and 4, and when it comes to plotting, that can create issues).

In [7]:
def aggregate(data):
    '''
    Aggregates each of the columns provided by response_col, for the chosen
    dataset. Uses the known possible values for each column, it counts
    occurences of each (inc. number missing) and makes the answer as a single
    dataframe row, where counts and percentages and categories are stored as
    lists within cells of that row. The function returns a dataframe containing
    all of those rows.
    Inputs:
    - data - dataframe
    '''
    # Initialise list to store rows of the dataframe
    rows = list()

    # Loop through the columns of interest
    for col_lab in response_col:

        # Find the name of the numeric version of the column
        col = col_lab.replace('_lab', '')

        # Find value counts
        value_counts = data[col].value_counts(dropna=False)

        # Get all possible category values and labels from dictionary
        cat = list(labels[col].keys())
        cat_lab = list(labels[col].values())

        # Initalise list for storing counts
        counts = []
        # For each of the possible values in labels - if the value was present,
        # extract from the counts series, but if not, set count to 0
        for value in labels[col].keys():
            if value in value_counts.index:
                counts.append(value_counts[value])
            else:
                counts.append(0)

        # Convert list of counts to list of percentages, and create rounded version
        percentages = [(x/sum(counts))*100 for x in counts]

        # Create dataframe row using the calculated data
        df_row = pd.DataFrame({
            'cat': [cat],
            'cat_lab': [cat_lab],
            'count': [counts],
            'percentage': [percentages],
            'measure': col,
            'n_responses': sum(counts)
        })
        # Append to list
        rows.append(df_row)

    # Combine into a single dataframe and return
    return(pd.concat(rows))


In [8]:
# Create version where every question has count 0, to use when a school has no
# pupils of a particular subgroup (i.e. no-one in certain FSM/SEN/gender/year)
no_pupils = aggregate(data)
no_pupils[['count', 'percentage', 'n_responses']] = 0
no_pupils.head()

,cat,cat_lab,count,percentage,measure,n_responses
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",0,0,autonomy_pressure,0
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",0,0,autonomy_express,0
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",0,0,autonomy_decide,0
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",0,0,autonomy_told,0
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",0,0,autonomy_myself,0


In [9]:
# Define the groups that we want to aggregate by - when providing a filter,
# the first value is the name of the category and the second is the variable
groups = [
    'All',
    ['Year 8', 'year_group_lab'],
    ['Year 10', 'year_group_lab'],
    ['Girl', 'gender_lab'],
    ['Boy', 'gender_lab'],
    ['FSM', 'fsm_lab'],
    ['Non-FSM', 'fsm_lab'],
    ['SEN', 'sen_lab'],
    ['Non-SEN', 'sen_lab']
]

In [10]:
# Initialise list to store results
result_list = list()

# For each of the schools (which we know will all be present at least once
# as we base the school list on the dataset itself)
schools = data['school_lab'].dropna().drop_duplicates().sort_values()
for school in schools:
    # For each the groupings
    for group in groups:

        # Find results for that school. If group is not equal to all,
        # then apply additional filters
        to_agg = data[data['school_lab'] == school]
        if group != 'All':
            to_agg = to_agg[to_agg[group[1]] == group[0]]

        # If the dataframe is empty (i.e. you applied a filter but there
        # were no students matching that filter) then set to the no_pupils df.
        # Otherwise, just aggregate the data
        if len(to_agg.index) == 0:
            res = no_pupils.copy()
        else:
            res = aggregate(to_agg)

        # Specify what school it was
        res['school_lab'] = school

        # Set each group as all, but replace the relevant one if filter used
        res['year_group_lab'] = 'All'
        res['gender_lab'] = 'All'
        res['fsm_lab'] = 'All'
        res['sen_lab'] = 'All'
        if group != 'All':
            res[group[1]] = group[0]

        # Append results to list
        result_list.append(res)

# Combine all the results into a single dataframe
result = pd.concat(result_list)

# Hide results where n<10
result.loc[result['n_responses'] < 10,
           ['count', 'percentage', 'n_responses']] = np.nan

In [11]:
# Preview head of dataframe
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[15, 22, 19, 19, 23, 17]","[13.043478260869565, 19.130434782608695, 16.52...",autonomy_pressure,115.0,School A,All,All,All,All
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[25, 25, 16, 24, 24, 1]","[21.73913043478261, 21.73913043478261, 13.9130...",autonomy_express,115.0,School A,All,All,All,All
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[24, 24, 15, 29, 21, 2]","[20.869565217391305, 20.869565217391305, 13.04...",autonomy_decide,115.0,School A,All,All,All,All
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[26, 20, 13, 27, 17, 12]","[22.608695652173914, 17.391304347826086, 11.30...",autonomy_told,115.0,School A,All,All,All,All
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[31, 20, 21, 19, 15, 9]","[26.956521739130434, 17.391304347826086, 18.26...",autonomy_myself,115.0,School A,All,All,All,All


In [12]:
# Preview school where I set there to be no SEN pupils
# Should show as NaN, as count was 0 which is <10
result[(result['school_lab'] == 'School B') & (result['sen_lab'] == 'SEN')]

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",NaN,NaN,autonomy_pressure,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",NaN,NaN,autonomy_express,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",NaN,NaN,autonomy_decide,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",NaN,NaN,autonomy_told,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...",NaN,NaN,autonomy_myself,NaN,School B,All,All,All,SEN
...,...,...,...,...,...,...,...,...,...,...,...
0,"[0, 1, nan]","[No, Yes, No response]",NaN,NaN,peer_talk,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, nan]","[Not at all, Slightly, Mostly, Fully, No respo...",NaN,NaN,peer_talk_listen,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, nan]","[Not helpful, Somewhat helpful, Very helpful, ...",NaN,NaN,peer_talk_helpful,NaN,School B,All,All,All,SEN
0,"[1, 2, 3, 4, nan]","[Very uncomfortable, Uncomfortable, Comfortabl...",NaN,NaN,peer_talk_if,NaN,School B,All,All,All,SEN


## Add groups for each measure

This uses add_keys() like `1_create_synthetic_data.ipynb`.

In [13]:
# Initialise dictionary of groups
groups = defaultdict(str)

# Define function for adding multiple keys with the same value
def add_keys(value, keys):
    '''
    Add multiple keys with the same value to the dictionary
    Inputs:
    value: String which is the value for all the keys
    keys: Array with the keys
    '''
    groups.update(dict.fromkeys(keys, value))

# Add the groups to the dictionary
add_keys('demographic', [
    'gender',
    'transgender',
    'sexual_orientation',
    'neurodivergent',
    'birth_parent1',
    'birth_parent2',
    'birth_you',
    'birth_you_age',
    'young_carer',
    'care_experience',
    'year_group',
    'fsm',
    'sen',
    'ethnicity',
    'english_additional'])

add_keys('autonomy', [
    'autonomy_pressure',
    'autonomy_express',
    'autonomy_decide',
    'autonomy_told',
    'autonomy_myself',
    'autonomy_choice'])
add_keys('life_satisfaction', ['life_satisfaction'])
add_keys('optimism', [
    'optimism_future',
    'optimism_best',
    'optimism_good',
    'optimism_work'])
add_keys('wellbeing', [
    'wellbeing_optimistic',
    'wellbeing_useful',
    'wellbeing_relaxed',
    'wellbeing_problems',
    'wellbeing_thinking',
    'wellbeing_close',
    'wellbeing_mind'])
add_keys('esteem', [
    'esteem_satisfied',
    'esteem_qualities',
    'esteem_well',
    'esteem_value',
    'esteem_good'])
add_keys('stress', [
    'stress_control',
    'stress_overcome',
    'stress_confident',
    'stress_way'])
add_keys('appearance', ['appearance_happy', 'appearance_feel'])
add_keys('negative', [
    'negative_lonely',
    'negative_unhappy',
    'negative_like',
    'negative_cry',
    'negative_school',
    'negative_worry',
    'negative_sleep',
    'negative_wake',
    'negative_shy',
    'negative_scared'])
add_keys('lonely', ['lonely'])
add_keys('support', ['support_ways', 'support_look'])

add_keys('sleep', ['sleep'])
add_keys('physical', ['physical_days', 'physical_hours'])
add_keys('free_like', ['free_like'])
add_keys('media', ['media_hours'])
add_keys('places', [
    'places_freq',
    'places_barriers___1',
    'places_barriers___2',
    'places_barriers___3',
    'places_barriers___4',
    'places_barriers___5',
    'places_barriers___6',
    'places_barriers___7',
    'places_barriers___8',
    'places_barriers___9'])
add_keys('school_belong', ['school_belong'])
add_keys('staff_relationship', [
    'staff_interest', 'staff_believe', 'staff_best', 'staff_listen'])

add_keys('talk', [
    'staff_talk', 'staff_talk_listen', 'staff_talk_helpful', 'staff_talk_if',
    'home_talk', 'home_talk_listen', 'home_talk_helpful', 'home_talk_if',
    'peer_talk', 'peer_talk_listen', 'peer_talk_helpful', 'peer_talk_if'])
add_keys('accept', [
    'accept_staff', 'accept_home', 'accept_local', 'accept_peer'])

add_keys('home_relationship', [
    'home_interest', 'home_believe', 'home_best', 'home_listen'])
add_keys('home_happy', ['home_happy'])

add_keys('local_env', ['local_safe', 'local_support', 'local_trust',
                       'local_neighbours', 'local_places'])
add_keys('discrim', ['discrim_race', 'discrim_gender', 'discrim_orientation',
                     'discrim_disability', 'discrim_faith'])
add_keys('belong_local', ['belong_local'])
add_keys('wealth', ['wealth'])
add_keys('future', ['future_options', 'future_interest', 'future_support'])
add_keys('climate', ['climate'])
add_keys('social', [
    'social_along', 'social_time', 'social_support', 'social_hard'])
add_keys('bully', ['bully_physical', 'bully_other', 'bully_cyber'])

# Preview the dictionary
groups

defaultdict(str,
            {'gender': 'demographic',
             'transgender': 'demographic',
             'sexual_orientation': 'demographic',
             'neurodivergent': 'demographic',
             'birth_parent1': 'demographic',
             'birth_parent2': 'demographic',
             'birth_you': 'demographic',
             'birth_you_age': 'demographic',
             'young_carer': 'demographic',
             'care_experience': 'demographic',
             'year_group': 'demographic',
             'fsm': 'demographic',
             'sen': 'demographic',
             'ethnicity': 'demographic',
             'english_additional': 'demographic',
             'autonomy_pressure': 'autonomy',
             'autonomy_express': 'autonomy',
             'autonomy_decide': 'autonomy',
             'autonomy_told': 'autonomy',
             'autonomy_myself': 'autonomy',
             'autonomy_choice': 'autonomy',
             'life_satisfaction': 'life_satisfaction',
             'opt

In [14]:
# Add groups and preview
result['group'] = result['measure'].map(groups)
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,group
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[15, 22, 19, 19, 23, 17]","[13.043478260869565, 19.130434782608695, 16.52...",autonomy_pressure,115.0,School A,All,All,All,All,autonomy
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[25, 25, 16, 24, 24, 1]","[21.73913043478261, 21.73913043478261, 13.9130...",autonomy_express,115.0,School A,All,All,All,All,autonomy
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[24, 24, 15, 29, 21, 2]","[20.869565217391305, 20.869565217391305, 13.04...",autonomy_decide,115.0,School A,All,All,All,All,autonomy
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[26, 20, 13, 27, 17, 12]","[22.608695652173914, 17.391304347826086, 11.30...",autonomy_told,115.0,School A,All,All,All,All,autonomy
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[31, 20, 21, 19, 15, 9]","[26.956521739130434, 17.391304347826086, 18.26...",autonomy_myself,115.0,School A,All,All,All,All,autonomy


## Add labels for each measure

In [15]:
# Define labels
labels = {
    'gender': 'Gender',
    'transgender': 'Do you consider yourself to be transgender?',
    'sexual_orientation': 'Sexual orientation',
    'neurodivergent': 'Do you identify as neurodivergent?',
    'autonomy_pressure': 'I feel pressured in my life',
    'autonomy_express': 'I generally feel free to express my ideas and opinions',
    'autonomy_decide': 'I feel like I am free to decide for myself how to live my life',
    'autonomy_told': 'In my daily life I often have to do what I am told',
    'autonomy_myself': 'I feel I can pretty much be myself in daily situations',
    'autonomy_choice': 'I have enough choice about how I spend my time',
    'life_satisfaction': 'Overall, how satisfied are you with your life nowadays?',
    'optimism_future': 'I am optimistic about my future',
    'optimism_best': 'In uncertain times, I expect the best',
    'optimism_good': 'I think good things are going to happen to me',
    'optimism_work': 'I believe that things will work out, no matter how difficult they seem',
    'wellbeing_optimistic': '''I've been feeling optimistic about the future''',
    'wellbeing_useful': '''I've been feeling useful''',
    'wellbeing_relaxed': '''I've been feeling relaxed''',
    'wellbeing_problems': '''I've been dealing with problems well''',
    'wellbeing_thinking': '''I've been thinking clearly''',
    'wellbeing_close': '''I've been feeling close to other people''',
    'wellbeing_mind': '''I've been able to make up my own mind about things''',
    'esteem_satisfied': 'On the whole, I am satisfied with myself',
    'esteem_qualities': 'I feel that I have a number of good qualities',
    'esteem_well': 'I am able to do things as well as most other people',
    'esteem_value': 'I am a person of value',
    'esteem_good': 'I feel good about myself',
    'stress_control': 'Felt you were unable to control the important things in your life',
    'stress_overcome': 'Felt that difficulties were piling up so high that you could not overcome them',
    'stress_confident': 'Felt confident about your ability to handle your personal problems',
    'stress_way': 'Felt that things were going your way',
    'appearance_happy': 'How happy are you with your appearance (the way that you look)?',
    'appearance_feel': 'My appearance affects how I feel about myself',
    'negative_lonely': 'I feel lonely',
    'negative_unhappy': 'I am unhappy',
    'negative_like': 'Nobody likes me',
    'negative_cry': 'I cry a lot',
    'negative_school': 'I worry when I am at school',
    'negative_worry': 'I worry a lot',
    'negative_sleep': 'I have problems sleeping',
    'negative_wake': 'I wake up in the night',
    'negative_shy': 'I am shy',
    'negative_scared': 'I feel scared',
    'lonely': 'How often do you feel lonely?',
    'support_ways': 'I have ways to support myself (e.g. to cope, or help myself feel better)',
    'support_look': 'I know where to look for advice on how to support myself',
    'sleep': 'Is the amount of sleep you normally get enough for you to feel awake and concentrate on your school work during the day?',
    'physical_days': 'How many days in a usual week are you physically active?',
    'physical_hours': 'How long on average do you spend being physically active?',
    'free_like': 'How often can you do things that you like in your free time?',
    'media_hours': 'On a normal weekday during term time, how much time do you spend on social media?',
    'places_freq': 'How many activities/places are there in your local area, that you choose to or would want to go to in your free time?',
    'places_barriers___1': '''There's nothing to do''',
    'places_barriers___2': '''I'm unable to get there and back''',
    'places_barriers___3': '''It's too expensive (to get there or take part)''',
    'places_barriers___4': 'Poor weather',
    'places_barriers___5': 'I have no-one to go with',
    'places_barriers___6': '''It's too busy''',
    'places_barriers___7': 'I feel uncomfortable/anxious about other people who might be there',
    'places_barriers___8': '''My parents/carers don't allow me to go''',
    'places_barriers___9': 'Other',
    'school_belong': 'I feel that I belong/belonged at my school',
    'staff_interest': 'Is interested in my schoolwork',
    'staff_believe': 'Believes that I will be a success',
    'staff_best': 'Wants me to do my best',
    'staff_listen': 'Listens to me when I have something to say',
    'staff_talk': 'An adult at school',
    'staff_talk_listen': 'An adult at school',
    'staff_talk_helpful': 'An adult at school',
    'staff_talk_if': 'An adult at school',
    'accept_staff': 'Adults at your school',
    'home_interest': 'Is interested in my schoolwork',
    'home_believe': 'Believes that I will be a success',
    'home_best': 'Wants me to do my best',
    'home_listen': 'Listens to me when I have something to say',
    'home_talk': 'One of your parents/carers',
    'home_talk_listen': 'One of your parents/carers',
    'home_talk_helpful': 'One of your parents/carers',
    'home_talk_if': 'One of your parents/carers',
    'accept_home': 'Your parents/carers',
    'home_happy': 'How happy are you with the home that you live in?',
    'young_carer': 'In the last year, have you regularly taken on caring responsibilities for a family member - e.g. due to illness, disability, mental health condition or drug/alcohol dependency?',
    'care_experience': 'Are you or have you ever been in care (living in a foster placement, residential placement, or private/kinship care)?',
    'local_safe': 'How safe do you feel when in your local area?',
    'local_support': 'People around here support each other with their wellbeing',
    'local_trust': 'You can trust people around here',
    'local_neighbours': 'I could ask for help or a favour from neighbours',
    'local_places': 'There are good places to spend your free time (e.g., leisure centres, parks, shops)',
    'discrim_race': 'Your race, skin colour or where you were born',
    'discrim_gender': 'Your gender',
    'discrim_orientation': 'Your sexual orientation',
    'discrim_disability': 'Disability',
    'discrim_faith': 'Your religion/faith',
    'belong_local': 'I feel like I belong in my local area',
    'accept_local': 'People in your local area',
    'wealth': 'Compared to your friends, is your family richer, poorer or about the same?',
    'future_options': 'How many options are available?',
    'future_interest': 'How do you feel about the options available?',
    'future_support': 'Do you feel (or think you would feel) supported to explore options that interest you, even if no-one else around you has done them before?',
    'climate': 'How often do you worry about the impact of climate change on your future?',
    'social_along': 'I get along with people around me',
    'social_time': 'People like to spend time with me',
    'social_support': 'I feel supported by my friends',
    'social_hard': 'My friends care about me when times are hard (for example if I am sick or have done something wrong)',
    'bully_physical': 'How often do you get physically bullied at school?',
    'bully_other': 'How often do you get bullied in other ways at school?',
    'bully_cyber': 'How often do you get cyber-bullied?',
    'peer_talk': 'Another person your age',
    'peer_talk_listen': 'Another person your age',
    'peer_talk_helpful': 'Another person your age',
    'peer_talk_if': 'Another person your age',
    'accept_peer': 'Other people your age'
}

In [16]:
# Add labels
result['measure_lab'] = result['measure'].map(labels)

# Preview
result.head()

,cat,cat_lab,count,percentage,measure,n_responses,school_lab,year_group_lab,gender_lab,fsm_lab,sen_lab,group,measure_lab
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[15, 22, 19, 19, 23, 17]","[13.043478260869565, 19.130434782608695, 16.52...",autonomy_pressure,115.0,School A,All,All,All,All,autonomy,I feel pressured in my life
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[25, 25, 16, 24, 24, 1]","[21.73913043478261, 21.73913043478261, 13.9130...",autonomy_express,115.0,School A,All,All,All,All,autonomy,I generally feel free to express my ideas and ...
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[24, 24, 15, 29, 21, 2]","[20.869565217391305, 20.869565217391305, 13.04...",autonomy_decide,115.0,School A,All,All,All,All,autonomy,I feel like I am free to decide for myself how...
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[26, 20, 13, 27, 17, 12]","[22.608695652173914, 17.391304347826086, 11.30...",autonomy_told,115.0,School A,All,All,All,All,autonomy,In my daily life I often have to do what I am ...
0,"[1, 2, 3, 4, 5, nan]","[1 - Completely not true, 2, 3, 4, 5 - Complet...","[31, 20, 21, 19, 15, 9]","[26.956521739130434, 17.391304347826086, 18.26...",autonomy_myself,115.0,School A,All,All,All,All,autonomy,I feel I can pretty much be myself in daily si...


## Save to csv

In [17]:
result.to_csv(os.path.join(paths.survey, paths.aggregate), index=False)